# Installation
Installing necessary modules/libraries

In [1]:
!pip install praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 4.2 MB/s eta 0:00:00


In [2]:
!pip install transformers wikipedia newspaper3k GoogleNews pyvis

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 63.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.1 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=5eea41fbb2731cd5ed232a1daa0d281c2fb59cc1f1d689ae215e3bed89a70752
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76e

# Data Collection

Steps to Scrape Reddit Data using PRAW:

In [3]:
import praw
import warnings
warnings.filterwarnings("ignore")

In [4]:
reddit = praw.Reddit(
    client_id='sdYz2l1izU63748uu_g9Bg',
    client_secret='HeIM71wklD4XzC8hpdPS9cYPCb4BWA',
    username='s15_j',
    password='*art*0215',
    user_agent='For scraping data for data mining project',
    check_for_async=False
)

In [5]:
subreddit = reddit.subreddit('mentalhealth')
top_posts = subreddit.top(limit=10)  # Fetch top 10 posts

for post in top_posts:
    print(post.author, " - ", post.title)


WarmlyEccentric  -  Mental Health Awareness Month: I have schizoaffective bipolar type and I just graduated with my BA in English (Magna Cum Laude) at age 28 after dropping out of high school and having intermittently relapsed into psychosis for a duration of 8 hospitalizations. With loving support success is possible
Alyndriel  -  Not to brag, but instead of laying in bed ALL day, I laid in bed for MOST of the day, then got up and took a shower. Next up, I conquer the world.
Crafty_n_depressed44  -  Thank you to the woman who rang me out at Target. You saved my life 😭❤️
starrfishandcoffee  -  PREACH
runawaylovebug  -  It’s been 2 months to the day since I last self harmed. I just want someone to be proud of me.
Emmengard  -  I made a suicide scale. Hoping it could help others.
notoriu5  -  Anyone else procrastinate going to sleep because they don’t want the next day to come, but then also get anxious about how many hours they have left before they need to get up?
aristotlesbitch4561  

# Data Preprocessing

To build a knowledge graph from text, we typically need to perform two steps:

1. Extract entities, a.k.a. Named Entity Recognition (NER), which are going to be the nodes of the knowledge graph.

2. Extract relations between the entities, a.k.a. Relation Classification (RC), which are going to be the edges of the knowledge graph.

These multiple-step pipelines often propagate errors or are limited to a small number of relation types. Recently, end-to-end approaches have been proposed to tackle both tasks simultaneously. This task is usually referred to as **Relation Extraction (RE)**. In this project, I will use an end-to-end model called REBEL, from the paper Relation Extraction By End-to-end Language generation.

In [6]:
# needed to load the REBEL model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import math
import torch

# wrapper for wikipedia API
import wikipedia

# scraping of web articles
from newspaper import Article, ArticleException

# google news scraping
from GoogleNews import GoogleNews

# graph visualization
from pyvis.network import Network

# show HTML in notebook
import IPython

Loading the Relation Extraction Model

In [7]:
# Load REBEL model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/344 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [8]:
def extract_relations_from_model_output(text):
  """
  This function is able to parse the strings generated by REBEL and transform
  them into relation triplets (e.g. the <Fabio, lives in, Italy> triplet).
  This function must take into account additional new tokens (i.e. the <triplet> ,
  <subj>, and <obj> tokens) used while training the model.
  This function is provided by the HuggingFace REBEL model card.

  Parameters
  -------------
  text (string): The textual data that will be processed

  Returns
  -------------
  list: Outputs a list of relations, where each relation is represented as a
        dictionary with the following keys:
          head: The subject of the relation (e.g. “Fabio”).
          type: The relation type (e.g. “lives in”).
          tail: The object of the relation (e.g. “Italy”).
  """
  relations = []
  relation, subject, relation, object_ = '', '', '', ''
  text = text.strip()
  current = 'x'
  text_replaced = text.replace("<s>", "").replace("<pad>", "").replace("</s>", "")
  for token in text_replaced.split():
      if token == "<triplet>":
          current = 't'
          if relation != '':
              relations.append({
                  'head': subject.strip(),
                  'type': relation.strip(),
                  'tail': object_.strip()
              })
              relation = ''
          subject = ''
      elif token == "<subj>":
          current = 's'
          if relation != '':
              relations.append({
                  'head': subject.strip(),
                  'type': relation.strip(),
                  'tail': object_.strip()
              })
          object_ = ''
      elif token == "<obj>":
          current = 'o'
          relation = ''
      else:
          if current == 't':
              subject += ' ' + token
          elif current == 's':
              object_ += ' ' + token
          elif current == 'o':
              relation += ' ' + token
  if subject != '' and relation != '' and object_ != '':
      relations.append({
          'head': subject.strip(),
          'type': relation.strip(),
          'tail': object_.strip()
      })
  return relations

 The KB class is made of a list of relations and has several methods to deal with adding new relations to the knowledge base or printing them.

In [28]:
class KB():
  """
  A class to represent knowledge base

  Attributes
  ----------
  relations : list
      first name of the person


  Methods
  -------
  are_relations_equal(r1, r2):
      Checks if two relations are equal or not

  exists_relation(r1):
      Checks if the relations already exists in the list of relations or not

  add_relation(r):
      Add a relation r to the list of relations

  print():
      Prints all the relations
  """
  def __init__(self):
      self.entities = {} # { entity_title: {...} }
      self.relations = [] # [ head: entity_title, type: ..., tail: entity_title,
        # meta: { article_url: { spans: [...] } } ]
      self.sources = {} # { article_url: {...} }

  def are_relations_equal(self, r1, r2):
      return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

  def exists_relation(self, r1):
      return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

  def add_relation(self, r):
      if not self.exists_relation(r):
          self.relations.append(r)

  def print(self):
      print("Relations:")
      for r in self.relations:
          print(f"  {r}")

  def merge_relations(self, r1):
      r2 = [r for r in self.relations
            if self.are_relations_equal(r1, r)][0]
      spans_to_add = [span for span in r1["meta"]["spans"]
                      if span not in r2["meta"]["spans"]]
      r2["meta"]["spans"] += spans_to_add

  def add_relation(self, r):
      if not self.exists_relation(r):
          self.relations.append(r)
      else:
          self.merge_relations(r)

  def get_wikipedia_data(self, candidate_entity):
      try:
          page = wikipedia.page(candidate_entity, auto_suggest=False)
          entity_data = {
              "title": page.title,
              "url": page.url,
              "summary": page.summary
          }
          return entity_data
      except:
          return None

  def add_entity(self, e):
      self.entities[e["title"]] = {k:v for k,v in e.items() if k != "title"}

  def add_relation(self, r):
      # check on wikipedia
      candidate_entities = [r["head"], r["tail"]]
      entities = [self.get_wikipedia_data(ent) for ent in candidate_entities]

      # if one entity does not exist, stop
      if any(ent is None for ent in entities):
          return

      # manage new entities
      for e in entities:
          self.add_entity(e)

      # rename relation entities with their wikipedia titles
      r["head"] = entities[0]["title"]
      r["tail"] = entities[1]["title"]

      # manage new relation
      if not self.exists_relation(r):
          self.relations.append(r)
      else:
          self.merge_relations(r)

  def print(self):
      print("Entities:")
      for e in self.entities.items():
          print(f"  {e}")
      print("Relations:")
      for r in self.relations:
          print(f"  {r}")

  def merge_relations(self, r2):
      r1 = [r for r in self.relations
            if self.are_relations_equal(r2, r)][0]

      # if different article
      article_url = list(r2["meta"].keys())[0]
      if article_url not in r1["meta"]:
          r1["meta"][article_url] = r2["meta"][article_url]

      # if existing article
      else:
          spans_to_add = [span for span in r2["meta"][article_url]["spans"]
                          if span not in r1["meta"][article_url]["spans"]]
          r1["meta"][article_url]["spans"] += spans_to_add


  def add_relation(self, r, article_title, article_publish_date):
      # check on wikipedia
      candidate_entities = [r["head"], r["tail"]]
      entities = [self.get_wikipedia_data(ent) for ent in candidate_entities]

      # if one entity does not exist, stop
      if any(ent is None for ent in entities):
          return

      # manage new entities
      for e in entities:
          self.add_entity(e)

      # rename relation entities with their wikipedia titles
      r["head"] = entities[0]["title"]
      r["tail"] = entities[1]["title"]

      # add source if not in kb
      article_url = list(r["meta"].keys())[0]
      if article_url not in self.sources:
          self.sources[article_url] = {
              "article_title": article_title,
              "article_publish_date": article_publish_date
          }

      # manage new relation
      if not self.exists_relation(r):
          self.relations.append(r)
      else:
          self.merge_relations(r)

  def print(self):
      print("Entities:")
      for e in self.entities.items():
          print(f"  {e}")
      print("Relations:")
      for r in self.relations:
          print(f"  {r}")
      print("Sources:")
      for s in self.sources.items():
          print(f"  {s}")

  def merge_with_kb(self, kb2):
      for r in kb2.relations:
          article_url = list(r["meta"].keys())[0]
          source_data = kb2.sources[article_url]
          self.add_relation(r, source_data["article_title"],
                            source_data["article_publish_date"])

Last, we define a *from_small_text_to_kb* function that returns a KB object with relations extracted from a short text. It does the following:

1. Initialize an empty knowledge base KB object.

2. Tokenize the input text.

3. Use REBEL to generate relations from the text.

4. Parse REBEL output and store relation triplets into the knowledge base object.

5. Return the knowledge base object.

In [10]:
# build a knowledge base from text
def from_small_text_to_kb(text, verbose=False):
    kb = KB()

    # Tokenizer text
    model_inputs = tokenizer(text, max_length=512, padding=True, truncation=True,
                            return_tensors='pt')
    if verbose:
        print(f"Num tokens: {len(model_inputs['input_ids'][0])}")

    # Generate
    gen_kwargs = {
        "max_length": 216,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": 3
    }
    generated_tokens = model.generate(
        **model_inputs,
        **gen_kwargs,
    )
    decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

    # create kb
    for sentence_pred in decoded_preds:
        relations = extract_relations_from_model_output(sentence_pred)
        for r in relations:
            kb.add_relation(r)

    return kb

Testing the above function on a small text

In [11]:
text = "Napoleon Bonaparte (born Napoleone di Buonaparte; 15 August 1769 – 5 " \
"May 1821), and later known by his regnal name Napoleon I, was a French military " \
"and political leader who rose to prominence during the French Revolution and led " \
"several successful campaigns during the Revolutionary Wars. He was the de facto " \
"leader of the French Republic as First Consul from 1799 to 1804. As Napoleon I, " \
"he was Emperor of the French from 1804 until 1814 and again in 1815. Napoleon's " \
"political and cultural legacy has endured, and he has been one of the most " \
"celebrated and controversial leaders in world history."

kb = from_small_text_to_kb(text, verbose=True)
kb.print()

Num tokens: 133
Relations:
  {'head': 'Napoleon Bonaparte', 'type': 'date of birth', 'tail': '15 August 1769'}
  {'head': 'Napoleon Bonaparte', 'type': 'date of death', 'tail': '5 May 1821'}
  {'head': 'Napoleon Bonaparte', 'type': 'participant in', 'tail': 'French Revolution'}
  {'head': 'Napoleon Bonaparte', 'type': 'conflict', 'tail': 'Revolutionary Wars'}
  {'head': 'Revolutionary Wars', 'type': 'part of', 'tail': 'French Revolution'}
  {'head': 'French Revolution', 'type': 'participant', 'tail': 'Napoleon Bonaparte'}
  {'head': 'Revolutionary Wars', 'type': 'participant', 'tail': 'Napoleon Bonaparte'}


### From Long Text to Knowledge Base

Transformer models like REBEL have memory requirements that grow quadratically with the size of the inputs. This means that REBEL is able to work on common hardware at a reasonable speed with inputs of about 512 tokens, which correspond to about 380 English words. However, we may need to extract relations from documents long several thousands of words.

Moreover, from my experiments with the model, it seems to work better with shorter inputs. Intuitively, raw text relations are often expressed in single or contiguous sentences, therefore it may not be necessary to consider a high number of sentences at the same time to extract specific relations. Additionally, extracting a few relations is a simpler task than extracting many relations.

So, how do we put all this together?

For example, we can divide an input text 1000 tokens long into eight shorter overlapping spans 128 tokens long and extract relations from each span. While doing so, we also add some metadata to the extracted relations containing their span boundaries. With this info, we are able to see from which span of the text we extracted a specific relation which is now saved in our knowledge base.

The class KB has been created in such a way so that span boundaries are saved as well. The relation dictionary has the keys:

1. head : The subject of the relation (e.g. “Fabio”).

2. type : The relation type (e.g. “lives in”).

3. tail : The object of the relation (e.g. “Italy”).

4. meta : A dictionary containing meta information about the relation. This dictionary has a spans key, whose value is the list of span boundaries (e.g. [[0, 128], [119, 247]]) where the relation has been found.

Next, we write the from_text_to_kb function, which is similar to the from_small_text_to_kb function but is able to manage longer texts by splitting them into spans. All the new code is about the spanning logic and the management of the spans into the relations.



In [17]:
def from_text_to_kb(text, span_length=128, verbose=False):
  '''
  Used to extract relations for each span and put them together in a knowledge base

  Parameters
  -------------
  text (string): The text that needs to be modified to KB
  span_length (int): Number representing the length of each span
  verbose (binary): Flag used to control whether additional information
                    or details are printed during execution

  Returns
  ------------
  binary_sum (str): Binary string of the sum of a and b
  '''
  # Tokenize whole text
  inputs = tokenizer([text], return_tensors="pt")

  # Compute span boundaries
  num_tokens = len(inputs["input_ids"][0])
  if verbose:
      print(f"Input has {num_tokens} tokens")
  num_spans = math.ceil(num_tokens / span_length)
  if verbose:
      print(f"Input has {num_spans} spans")
  overlap = math.ceil((num_spans * span_length - num_tokens) /
                      max(num_spans - 1, 1))
  spans_boundaries = []
  start = 0
  for i in range(num_spans):
      spans_boundaries.append([start + span_length * i,
                                start + span_length * (i + 1)])
      start -= overlap
  if verbose:
      print(f"Span boundaries are {spans_boundaries}")

  # Transform input with spans
  tensor_ids = [inputs["input_ids"][0][boundary[0]:boundary[1]]
                for boundary in spans_boundaries]
  tensor_masks = [inputs["attention_mask"][0][boundary[0]:boundary[1]]
                  for boundary in spans_boundaries]
  inputs = {
      "input_ids": torch.stack(tensor_ids),
      "attention_mask": torch.stack(tensor_masks)
  }

  # Generate relations
  num_return_sequences = 3
  gen_kwargs = {
      "max_length": 256,
      "length_penalty": 0,
      "num_beams": 3,
      "num_return_sequences": num_return_sequences
  }
  generated_tokens = model.generate(
      **inputs,
      **gen_kwargs,
  )

  # Decode relations
  decoded_preds = tokenizer.batch_decode(generated_tokens,
                                          skip_special_tokens=False)

  # Create knowledge base
  kb = KB()
  i = 0
  for sentence_pred in decoded_preds:
      current_span_index = i // num_return_sequences
      relations = extract_relations_from_model_output(sentence_pred)
      for relation in relations:
          relation["meta"] = {
              "spans": [spans_boundaries[current_span_index]]
          }
          kb.add_relation(relation)
      i += 1

  return kb

Let’s try it with a longer text of 726 tokens about Napoleon. We are currently splitting the text into spans long 128 tokens.

In [13]:
text = """
Napoleon Bonaparte (born Napoleone di Buonaparte; 15 August 1769 – 5 May 1821), and later known by his regnal name Napoleon I, was a French military and political leader who rose to prominence during the French Revolution and led several successful campaigns during the Revolutionary Wars. He was the de facto leader of the French Republic as First Consul from 1799 to 1804. As Napoleon I, he was Emperor of the French from 1804 until 1814 and again in 1815. Napoleon's political and cultural legacy has endured, and he has been one of the most celebrated and controversial leaders in world history. Napoleon was born on the island of Corsica not long after its annexation by the Kingdom of France.[5] He supported the French Revolution in 1789 while serving in the French army, and tried to spread its ideals to his native Corsica. He rose rapidly in the Army after he saved the governing French Directory by firing on royalist insurgents. In 1796, he began a military campaign against the Austrians and their Italian allies, scoring decisive victories and becoming a national hero. Two years later, he led a military expedition to Egypt that served as a springboard to political power. He engineered a coup in November 1799 and became First Consul of the Republic. Differences with the British meant that the French faced the War of the Third Coalition by 1805. Napoleon shattered this coalition with victories in the Ulm Campaign, and at the Battle of Austerlitz, which led to the dissolving of the Holy Roman Empire. In 1806, the Fourth Coalition took up arms against him because Prussia became worried about growing French influence on the continent. Napoleon knocked out Prussia at the battles of Jena and Auerstedt, marched the Grande Armée into Eastern Europe, annihilating the Russians in June 1807 at Friedland, and forcing the defeated nations of the Fourth Coalition to accept the Treaties of Tilsit. Two years later, the Austrians challenged the French again during the War of the Fifth Coalition, but Napoleon solidified his grip over Europe after triumphing at the Battle of Wagram. Hoping to extend the Continental System, his embargo against Britain, Napoleon invaded the Iberian Peninsula and declared his brother Joseph King of Spain in 1808. The Spanish and the Portuguese revolted in the Peninsular War, culminating in defeat for Napoleon's marshals. Napoleon launched an invasion of Russia in the summer of 1812. The resulting campaign witnessed the catastrophic retreat of Napoleon's Grande Armée. In 1813, Prussia and Austria joined Russian forces in a Sixth Coalition against France. A chaotic military campaign resulted in a large coalition army defeating Napoleon at the Battle of Leipzig in October 1813. The coalition invaded France and captured Paris, forcing Napoleon to abdicate in April 1814. He was exiled to the island of Elba, between Corsica and Italy. In France, the Bourbons were restored to power. However, Napoleon escaped Elba in February 1815 and took control of France.[6][7] The Allies responded by forming a Seventh Coalition, which defeated Napoleon at the Battle of Waterloo in June 1815. The British exiled him to the remote island of Saint Helena in the Atlantic, where he died in 1821 at the age of 51. Napoleon had an extensive impact on the modern world, bringing liberal reforms to the many countries he conquered, especially the Low Countries, Switzerland, and parts of modern Italy and Germany. He implemented liberal policies in France and Western Europe.
"""

kb = from_text_to_kb(text, verbose=True)
kb.print()

Input has 726 tokens
Input has 6 spans
Span boundaries are [[0, 128], [119, 247], [238, 366], [357, 485], [476, 604], [595, 723]]
Relations:
  {'head': 'Napoleon Bonaparte', 'type': 'date of birth', 'tail': '15 August 1769', 'meta': {'spans': [[0, 128]]}}
  {'head': 'Napoleon Bonaparte', 'type': 'date of death', 'tail': '5 May 1821', 'meta': {'spans': [[0, 128]]}}
  {'head': 'Napoleon Bonaparte', 'type': 'participant in', 'tail': 'French Revolution', 'meta': {'spans': [[0, 128]]}}
  {'head': 'Napoleon Bonaparte', 'type': 'conflict', 'tail': 'Revolutionary Wars', 'meta': {'spans': [[0, 128]]}}
  {'head': 'Revolutionary Wars', 'type': 'part of', 'tail': 'French Revolution', 'meta': {'spans': [[0, 128]]}}
  {'head': 'French Revolution', 'type': 'participant', 'tail': 'Napoleon Bonaparte', 'meta': {'spans': [[0, 128]]}}
  {'head': 'Revolutionary Wars', 'type': 'participant', 'tail': 'Napoleon Bonaparte', 'meta': {'spans': [[0, 128]]}}
  {'head': 'French Revolution', 'type': 'country', 'tai

The text has been split into six spans, from which 23 relations have been extracted! Note that we also know from which text span each relation comes.



### Entity Linking

If you look closely at the extracted relations, you can see that there’s a relation with the entity “Napoleon Bonaparte” and a relation with the entity “Napoleon”. How can we tell our knowledge base that the two entities should be treated as the same?

One way to do this is to use the wikipedia library to check if “Napoleon Bonaparte” and “Napoleon” have the same Wikipedia page. If so, they are normalized to the title of the Wikipedia page. If an extracted entity doesn’t have a corresponding Wikipedia page, we ignore it at the moment. This step is commonly called Entity Linking.

Note that this approach relies on Wikipedia to be constantly updated by people with relevant entities. Therefore, it won’t work if you want to extract entities different from the ones already present in Wikipedia. Moreover, note that we are ignoring “date” (e.g. the 15 August 1769 in <Napoleon, date of birth, 15 August 1769>) entities for simplicity.

The KB has been modified to store an entities dictionary with the entities of the stored relations. The keys are the entity identifiers (i.e. the title of the corresponding Wikipedia page), and the value is a dictionary containing the Wikipedia page url and its summary. When adding a new relation, we now check its entities with the wikipedia library.

In [20]:
text = """
Napoleon Bonaparte (born Napoleone di Buonaparte; 15 August 1769 – 5 May 1821), and later known by his regnal name Napoleon I, was a French military and political leader who rose to prominence during the French Revolution and led several successful campaigns during the Revolutionary Wars. He was the de facto leader of the French Republic as First Consul from 1799 to 1804. As Napoleon I, he was Emperor of the French from 1804 until 1814 and again in 1815. Napoleon's political and cultural legacy has endured, and he has been one of the most celebrated and controversial leaders in world history. Napoleon was born on the island of Corsica not long after its annexation by the Kingdom of France.[5] He supported the French Revolution in 1789 while serving in the French army, and tried to spread its ideals to his native Corsica. He rose rapidly in the Army after he saved the governing French Directory by firing on royalist insurgents. In 1796, he began a military campaign against the Austrians and their Italian allies, scoring decisive victories and becoming a national hero. Two years later, he led a military expedition to Egypt that served as a springboard to political power. He engineered a coup in November 1799 and became First Consul of the Republic. Differences with the British meant that the French faced the War of the Third Coalition by 1805. Napoleon shattered this coalition with victories in the Ulm Campaign, and at the Battle of Austerlitz, which led to the dissolving of the Holy Roman Empire. In 1806, the Fourth Coalition took up arms against him because Prussia became worried about growing French influence on the continent. Napoleon knocked out Prussia at the battles of Jena and Auerstedt, marched the Grande Armée into Eastern Europe, annihilating the Russians in June 1807 at Friedland, and forcing the defeated nations of the Fourth Coalition to accept the Treaties of Tilsit. Two years later, the Austrians challenged the French again during the War of the Fifth Coalition, but Napoleon solidified his grip over Europe after triumphing at the Battle of Wagram. Hoping to extend the Continental System, his embargo against Britain, Napoleon invaded the Iberian Peninsula and declared his brother Joseph King of Spain in 1808. The Spanish and the Portuguese revolted in the Peninsular War, culminating in defeat for Napoleon's marshals. Napoleon launched an invasion of Russia in the summer of 1812. The resulting campaign witnessed the catastrophic retreat of Napoleon's Grande Armée. In 1813, Prussia and Austria joined Russian forces in a Sixth Coalition against France. A chaotic military campaign resulted in a large coalition army defeating Napoleon at the Battle of Leipzig in October 1813. The coalition invaded France and captured Paris, forcing Napoleon to abdicate in April 1814. He was exiled to the island of Elba, between Corsica and Italy. In France, the Bourbons were restored to power. However, Napoleon escaped Elba in February 1815 and took control of France.[6][7] The Allies responded by forming a Seventh Coalition, which defeated Napoleon at the Battle of Waterloo in June 1815. The British exiled him to the remote island of Saint Helena in the Atlantic, where he died in 1821 at the age of 51. Napoleon had an extensive impact on the modern world, bringing liberal reforms to the many countries he conquered, especially the Low Countries, Switzerland, and parts of modern Italy and Germany. He implemented liberal policies in France and Western Europe.
"""

kb = from_text_to_kb(text)
kb.print()


Entities:
  ('Napoleon', {'url': 'https://en.wikipedia.org/wiki/Napoleon', 'summary': "Napoleon Bonaparte (born Napoleone Buonaparte; 15 August 1769 – 5 May 1821), later known by his regnal name Napoleon I, was a French emperor and military commander who rose to prominence during the French Revolution and led successful campaigns during the Revolutionary Wars. He was the leader of the French Republic as First Consul from 1799 to 1804, then of the French Empire as Emperor of the French from 1804 until 1814, and briefly again in 1815. His political and cultural legacy endures as a celebrated and controversial leader. He initiated many enduring reforms, but has been criticized for his authoritarian rule. He is considered one of the greatest military commanders in history and his wars and campaigns are still studied at military schools worldwide. However, historians still debate whether he was responsible for the Napoleonic Wars in which between three and six million people died.Napoleon w

All the extracted entities are linked to Wikipedia pages and normalized with their titles. “Napoleon Bonaparte” and “Napoleon” are now both referred to with “Napoleon”!

### Extract KB from Web Articles

We want the knowledge base to manage the addition of relations and entities from articles from around the web, and to keep track of where each relation comes from.

To do this, we need to modify our KB class so that:

Along with relations and entities, sources (i.e. articles from around the web) are stored as well. Each article has its URL as key and a dictionary with keys article_title and article_publish_date as value. We’ll see later how to extract these two features.

When we add a new relation to our knowledge base, the relation meta field is now a dictionary with article URLs as keys, and another dictionary containing the spans as value. In this way, the knowledge base keeps track of all the articles from which a specific relation has been extracted. This information can be an indicator of the quality of an extracted relation.

The 'from_text_to_kb' function is modified so that it prepares the relation meta field taking into account article URLs as well.

In [23]:
def from_text_to_kb(text, article_url, span_length=128, article_title=None,
                    article_publish_date=None, verbose=False):
  '''
  Used to extract relations for each span and put them together in a knowledge base

  Parameters
  -------------
  text (string): The text that needs to be modified to KB
  span_length (int): Number representing the length of each span
  verbose (binary): Flag used to control whether additional information
                    or details are printed during execution

  Returns
  ------------
  binary_sum (str): Binary string of the sum of a and b
  '''
  # Tokenize whole text
  inputs = tokenizer([text], return_tensors="pt")

  # Compute span boundaries
  num_tokens = len(inputs["input_ids"][0])
  if verbose:
      print(f"Input has {num_tokens} tokens")
  num_spans = math.ceil(num_tokens / span_length)
  if verbose:
      print(f"Input has {num_spans} spans")
  overlap = math.ceil((num_spans * span_length - num_tokens) /
                      max(num_spans - 1, 1))
  spans_boundaries = []
  start = 0
  for i in range(num_spans):
      spans_boundaries.append([start + span_length * i,
                                start + span_length * (i + 1)])
      start -= overlap
  if verbose:
      print(f"Span boundaries are {spans_boundaries}")

  # Transform input with spans
  tensor_ids = [inputs["input_ids"][0][boundary[0]:boundary[1]]
                for boundary in spans_boundaries]
  tensor_masks = [inputs["attention_mask"][0][boundary[0]:boundary[1]]
                  for boundary in spans_boundaries]
  inputs = {
      "input_ids": torch.stack(tensor_ids),
      "attention_mask": torch.stack(tensor_masks)
  }

  # Generate relations
  num_return_sequences = 3
  gen_kwargs = {
      "max_length": 256,
      "length_penalty": 0,
      "num_beams": 3,
      "num_return_sequences": num_return_sequences
  }
  generated_tokens = model.generate(
      **inputs,
      **gen_kwargs,
  )

  # Decode relations
  decoded_preds = tokenizer.batch_decode(generated_tokens,
                                          skip_special_tokens=False)

  # create kb
  kb = KB()
  i = 0
  for sentence_pred in decoded_preds:
      current_span_index = i // num_return_sequences
      relations = extract_relations_from_model_output(sentence_pred)
      for relation in relations:
          relation["meta"] = {
              article_url: {
                  "spans": [spans_boundaries[current_span_index]]
              }
          }
          kb.add_relation(relation, article_title, article_publish_date)
      i += 1


  return kb

We use the newspaper library to download and parse articles from URLs and define a from_url_to_kb function. The library automatically extracts the article text, title, and publish date (if present).

In [24]:
# parse an article with newspaper3k
def get_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article

# extract the article from the url (along with metadata), extract relations and populate a KB
def from_url_to_kb(url):
    article = get_article(url)
    config = {
        "article_title": article.title,
        "article_publish_date": article.publish_date
    }
    kb = from_text_to_kb(article.text, article.url, **config)
    return kb

Let’s try to extract a knowledge base from the article Microstrategy chief: ‘[Bitcoin is going to go into the millions](https://finance.yahoo.com/news/microstrategy-bitcoin-millions-142143795.html?guccounter=1&guce_referrer=aHR0cHM6Ly93d3cubmxwbGFuZXQub3JnLw&guce_referrer_sig=AQAAAKLiKHo8mSFXWq85FN9wt-Gjvxzrx4SyAlWMjz8rt5oOEl5tDD86TUgoYLSFYLwlHrLP4ROLc2mJuGrF-Xhd_SkaChbMRU0bKyLdwhKi5DX2ITPOs6e5l7Tdt4w-MKeIYPWdYY7xeL85gniT-UGb_BvBaFsItdlADUDJkkCOCIn_)’.

In [27]:
# test the `from_url_to_kb` function
url = "https://finance.yahoo.com/news/microstrategy-bitcoin-millions-142143795.html"
kb = from_url_to_kb(url)
kb.print()

Entities:
  ('MicroStrategy', {'url': 'https://en.wikipedia.org/wiki/MicroStrategy', 'summary': "MicroStrategy Incorporated is an American company that provides business intelligence (BI), mobile software, and cloud-based services. Founded in 1989 by Michael J. Saylor, Sanju Bansal, and Thomas Spahr, the firm develops software to analyze internal and external data in order to make business decisions and to develop mobile apps. It is a public company headquartered in Tysons Corner, Virginia, in the Washington metropolitan area. Its primary business analytics competitors include SAP AG Business Objects, IBM Cognos, and Oracle Corporation's BI Platform. Saylor is the Executive Chairman and, from 1989 to 2022, was the CEO."})
  ('Michael J. Saylor', {'url': 'https://en.wikipedia.org/wiki/Michael_J._Saylor', 'summary': "Michael J. Saylor (born February 4, 1965) is an American entrepreneur and business executive. He is the executive chairman and co-founder of MicroStrategy, a company that pr

The KB is showing a lot of information!

1. From the **entities** list, we see that Microstrategy is an American company.

2. From the **relations** list, we see that Michael J. Saylor is a founder of the Microstrategy company, and where we extracted such relation (i.e. the article URL and the text span).

3. From the **sources** list, we see the title and publish date of the aforementioned article.

### Google News: Extract KB from Multiple Articles
Consider this last scenario: creating a knowledge base from multiple articles. We can deal with it by extracting a separate knowledge base from each article and then merging all the knowledge bases together. A 'merge_with_kb' method to our KB class has been added above.

Then, we use the GoogleNews library to get the URLs of recent news articles about a specific topic. Once we have multiple URLs, we feed them to the from_urls_to_kb function, which extracts a knowledge base from each article and then merges them together.

In [29]:
# get news links from google news
def get_news_links(query, lang="en", region="US", pages=1, max_links=100000):
    googlenews = GoogleNews(lang=lang, region=region)
    googlenews.search(query)
    all_urls = []
    for page in range(pages):
        googlenews.get_page(page)
        all_urls += googlenews.get_links()
    return list(set(all_urls))[:max_links]

# build a KB from multiple news links
def from_urls_to_kb(urls, verbose=False):
    kb = KB()
    if verbose:
        print(f"{len(urls)} links to visit")
    for url in urls:
        if verbose:
            print(f"Visiting {url}...")
        try:
            kb_url = from_url_to_kb(url)
            kb.merge_with_kb(kb_url)
        except ArticleException:
            if verbose:
                print(f"  Couldn't download article at url {url}")
    return kb

In [32]:
# test the `from_urls_to_kb` function
news_links = get_news_links("OpenAI", pages=1, max_links=10)
kb = from_urls_to_kb(news_links, verbose=True)
kb.print()

10 links to visit
Visiting https://www.theguardian.com/commentisfree/2023/nov/26/artificial-intelligence-harm-worry-about-people-control-openai&ved=2ahUKEwj1mKTbjOOCAxX8uYkEHStkDm8QxfQBegQIBRAC&usg=AOvVaw0MsHtAoGNPPaI4hw1ioP80...
  Couldn't download article at url https://www.theguardian.com/commentisfree/2023/nov/26/artificial-intelligence-harm-worry-about-people-control-openai&ved=2ahUKEwj1mKTbjOOCAxX8uYkEHStkDm8QxfQBegQIBRAC&usg=AOvVaw0MsHtAoGNPPaI4hw1ioP80
Visiting https://www.theglobeandmail.com/business/article-sam-altman-openai-livable-cities-rents-november-26/&ved=2ahUKEwj1mKTbjOOCAxX8uYkEHStkDm8QxfQBegQICBAC&usg=AOvVaw10cKtupTGDFWLEWzWUwN6J...
  Couldn't download article at url https://www.theglobeandmail.com/business/article-sam-altman-openai-livable-cities-rents-november-26/&ved=2ahUKEwj1mKTbjOOCAxX8uYkEHStkDm8QxfQBegQICBAC&usg=AOvVaw10cKtupTGDFWLEWzWUwN6J
Visiting https://www.ft.com/content/c4dd2ec0-026f-45dd-a7d4-d7aaa5cf9396&ved=2ahUKEwj1mKTbjOOCAxX8uYkEHStkDm8QxfQBegQIAR

Some articles couldn't be downloaded because they are paid articles and need subscription to access them.
We can see that the knowledge base is getting bigger, containing 13 entities, 8 relations and 3 sources

### Visualize KB

Let’s visualize the output of our work by plotting the knowledge bases. As our knowledge bases are graphs, we can use the pyvis library, which allows the creation of interactive network visualizations.

We define a save_network_html function that:

1. Initialize an empty directed pyvis network.

2. Add the knowledge base entities as nodes.

3. Add the knowledge base relations as edges.

4. Save the network in an HTML file.

In [41]:
# from KB to HTML visualization
def save_network_html(kb, filename="network.html"):
  # create network
  net = Network(directed=True, width="auto", height="700px", bgcolor="#eeeeee")

  # nodes
  color_entity = "#00FF00"
  for e in kb.entities:
      net.add_node(e, shape="circle", color=color_entity)

  # edges
  for r in kb.relations:
      net.add_edge(r["head"], r["tail"],
                  title=r["type"], label=r["type"])

  # print(net)

  # save network
  net.repulsion(
      node_distance=200,
      central_gravity=0.2,
      spring_length=200,
      spring_strength=0.05,
      damping=0.09
  )
  net.set_edge_smooth('dynamic')

  try:
    net.show(filename, notebook=False)
  except Exception as e:
    print(f"Error during visualization: {e}")


Let’s try the save_network_html function with a knowledge base built from 20 news articles about OpenAI.

In [43]:
# extract KB from news about OpenAI and visualize it
news_links = get_news_links("OpenAI", pages=5, max_links=20)
kb = from_urls_to_kb(news_links, verbose=True)
filename = "network_3_google.html"
save_network_html(kb, filename=filename)

20 links to visit
Visiting https://www.forbes.com/sites/lanceeliot/2023/11/26/about-that-mysterious-ai-breakthrough-known-as-q-by-openai-that-allegedly-attains-true-ai-or-is-on-the-path-toward-artificial-general-intelligence-agi/&ved=2ahUKEwiR4qDGlOOCAxVjjokEHS90AmYQxfQBegQIBBAC&usg=AOvVaw1ezDUYHMfdMz3SFiCRzgwg...
  Couldn't download article at url https://www.forbes.com/sites/lanceeliot/2023/11/26/about-that-mysterious-ai-breakthrough-known-as-q-by-openai-that-allegedly-attains-true-ai-or-is-on-the-path-toward-artificial-general-intelligence-agi/&ved=2ahUKEwiR4qDGlOOCAxVjjokEHS90AmYQxfQBegQIBBAC&usg=AOvVaw1ezDUYHMfdMz3SFiCRzgwg
Visiting https://www.syracuse.com/opinion/2023/11/editorial-cartoons-for-nov-26-2023-open-ai-meltdown-bidens-birthday-thanksgiving.html&ved=2ahUKEwjWj7_GlOOCAxWck4kEHTIqDrwQxfQBegQIBhAC&usg=AOvVaw2CkvnaoZ6yJSYFBAo_SrFj...
Visiting https://www.syracuse.com/opinion/2023/11/editorial-cartoons-for-nov-26-2023-open-ai-meltdown-bidens-birthday-thanksgiving.html&ved=2

### Rendering the HTML file in Colab

In [58]:
from IPython.display import HTML

# Specify the file name you uploaded
file_name = 'network_3_google.html'

# Read the HTML file content
with open(file_name, 'r') as file:
    html_content = file.read()

# Display the HTML content
HTML(html_content)


## Future Scope

This project can be further worked upon where link prediction can be done using TransE model in PyG library. We can also expand this project to use datasets from other forums to analyse the relationships between entities.